In [ ]:
"""
=================================================================
ANÁLISIS EXPLORATORIO DE DATOS - DETECCIÓN DE EXOPLANETAS
NASA Space Apps Challenge
=================================================================
Este notebook te ayudará a explorar y entender los datasets de:
- Kepler Objects of Interest (KOI)
- K2 Planets and Candidates
- TESS Objects of Interest (TOI)
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("=" * 70)
print("CARGANDO DATASETS DE EXOPLANETAS")
print("=" * 70)

# =================================================================
# 1. CARGA DE DATOS
# =================================================================

# Carga los tres datasets
kepler_df = pd.read_csv('cumulative_2025.10.01_09.10.49.csv')
k2_df = pd.read_csv('k2pandc_2025.10.01_09.14.10.csv')
tess_df = pd.read_csv('TOI_2025.10.01_09.13.47.csv')

print(f"\n✓ Kepler KOI cargado: {kepler_df.shape[0]} filas, {kepler_df.shape[1]} columnas")
print(f"✓ K2 cargado: {k2_df.shape[0]} filas, {k2_df.shape[1]} columnas")
print(f"✓ TESS TOI cargado: {tess_df.shape[0]} filas, {tess_df.shape[1]} columnas")

# =================================================================
# 2. EXPLORACIÓN INICIAL - KEPLER KOI
# =================================================================

print("\n" + "=" * 70)
print("DATASET 1: KEPLER OBJECTS OF INTEREST (KOI)")
print("=" * 70)

print("\n📊 INFORMACIÓN GENERAL:")
print(kepler_df.info())

print("\n🎯 COLUMNA OBJETIVO: koi_disposition")
print(kepler_df['koi_disposition'].value_counts())
print("\nDistribución porcentual:")
print(kepler_df['koi_disposition'].value_counts(normalize=True) * 100)

print("\n📋 PRIMERAS 5 FILAS:")
display(kepler_df.head())

print("\n📈 ESTADÍSTICAS DESCRIPTIVAS:")
display(kepler_df.describe())

# =================================================================
# 3. ANÁLISIS DE VALORES NULOS - KEPLER
# =================================================================

print("\n" + "=" * 70)
print("ANÁLISIS DE VALORES NULOS - KEPLER")
print("=" * 70)

# Calcular porcentaje de nulos por columna
null_counts = kepler_df.isnull().sum()
null_percentage = (null_counts / len(kepler_df)) * 100
null_df = pd.DataFrame({
    'Columna': null_counts.index,
    'Valores_Nulos': null_counts.values,
    'Porcentaje': null_percentage.values
})
null_df = null_df[null_df['Valores_Nulos'] > 0].sort_values('Porcentaje', ascending=False)

print(f"\n📊 Total de columnas con valores nulos: {len(null_df)}")
print("\n🔝 Top 20 columnas con más valores nulos:")
display(null_df.head(20))

# Visualización de nulos
plt.figure(figsize=(12, 6))
top_nulls = null_df.head(20)
plt.barh(top_nulls['Columna'], top_nulls['Porcentaje'])
plt.xlabel('Porcentaje de Valores Nulos (%)')
plt.title('Top 20 Columnas con Más Valores Nulos - Kepler KOI')
plt.tight_layout()
plt.show()

# =================================================================
# 4. IDENTIFICACIÓN DE COLUMNAS RELEVANTES - KEPLER
# =================================================================

print("\n" + "=" * 70)
print("IDENTIFICACIÓN DE COLUMNAS RELEVANTES - KEPLER")
print("=" * 70)

# Categorías de columnas según el paper
print("\n📚 CATEGORIZACIÓN DE COLUMNAS:\n")

# Identificadores (NO útiles para ML)
id_columns = [col for col in kepler_df.columns if any(x in col.lower() for x in ['kepid', 'kepoi', 'rowid', 'name'])]
print(f"🔸 Columnas de IDENTIFICACIÓN (eliminar): {len(id_columns)}")
print(id_columns[:10], "...\n")

# Columnas objetivo y scores (NO usar como features)
target_columns = [col for col in kepler_df.columns if any(x in col.lower() for x in ['disposition', 'score'])]
print(f"🎯 Columnas OBJETIVO/SCORE (no usar como features): {len(target_columns)}")
print(target_columns, "\n")

# Parámetros orbitales (MUY importantes)
orbital_columns = [col for col in kepler_df.columns if any(x in col.lower() for x in ['period', 'epoch', 'duration', 'depth', 'impact', 'ror', 'dor'])]
print(f"🌍 Parámetros ORBITALES (muy importantes): {len(orbital_columns)}")
print(orbital_columns[:15], "...\n")

# Parámetros estelares (importantes)
stellar_columns = [col for col in kepler_df.columns if any(x in col.lower() for x in ['steff', 'slogg', 'srad', 'smass', 'smet'])]
print(f"⭐ Parámetros ESTELARES (importantes): {len(stellar_columns)}")
print(stellar_columns[:15], "...\n")

# Parámetros planetarios (muy importantes)
planet_columns = [col for col in kepler_df.columns if any(x in col.lower() for x in ['prad', 'teq', 'insol', 'dor'])]
print(f"🪐 Parámetros PLANETARIOS (muy importantes): {len(planet_columns)}")
print(planet_columns[:15], "...\n")

# Columnas de error (considerar eliminar)
error_columns = [col for col in kepler_df.columns if 'err' in col.lower()]
print(f"⚠️  Columnas de ERROR (considerar eliminar): {len(error_columns)}")

# =================================================================
# 5. ANÁLISIS DE CORRELACIÓN - KEPLER
# =================================================================

print("\n" + "=" * 70)
print("ANÁLISIS DE CORRELACIÓN - KEPLER")
print("=" * 70)

# Seleccionar solo columnas numéricas (sin errores ni IDs)
numeric_cols = kepler_df.select_dtypes(include=[np.number]).columns
cols_to_analyze = [col for col in numeric_cols if not any(x in col.lower() for x in ['err', 'kepid', 'rowid', 'score', 'disposition'])]

# Calcular matriz de correlación
correlation_matrix = kepler_df[cols_to_analyze].corr()

# Visualizar correlación
plt.figure(figsize=(14, 12))
sns.heatmap(correlation_matrix, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación - Variables Numéricas (Kepler KOI)', fontsize=14)
plt.tight_layout()
plt.show()

# =================================================================
# 6. ANÁLISIS RÁPIDO K2
# =================================================================

print("\n" + "=" * 70)
print("DATASET 2: K2 PLANETS AND CANDIDATES")
print("=" * 70)

print("\n📊 Dimensiones:", k2_df.shape)
print("\n🎯 Columna objetivo: pl_def_disposition (Archive Disposition)")
if 'pl_def_disposition' in k2_df.columns:
    print(k2_df['pl_def_disposition'].value_counts())
else:
    # Buscar columna de disposición
    disp_cols = [col for col in k2_df.columns if 'disp' in col.lower()]
    print(f"Columnas de disposición encontradas: {disp_cols}")
    if disp_cols:
        print(k2_df[disp_cols[0]].value_counts())

print("\n📋 Columnas:")
print(k2_df.columns.tolist())

print("\n📊 Valores nulos:")
k2_nulls = (k2_df.isnull().sum() / len(k2_df) * 100).sort_values(ascending=False).head(10)
print(k2_nulls)

# =================================================================
# 7. ANÁLISIS RÁPIDO TESS
# =================================================================

print("\n" + "=" * 70)
print("DATASET 3: TESS OBJECTS OF INTEREST (TOI)")
print("=" * 70)

print("\n📊 Dimensiones:", tess_df.shape)
print("\n🎯 Columna objetivo: tfopwg_disp (TFOPWG Disposition)")
if 'tfopwg_disp' in tess_df.columns:
    print(tess_df['tfopwg_disp'].value_counts())
else:
    # Buscar columna de disposición
    disp_cols = [col for col in tess_df.columns if 'disp' in col.lower()]
    print(f"Columnas de disposición encontradas: {disp_cols}")
    if disp_cols:
        print(tess_df[disp_cols[0]].value_counts())

print("\n📋 Columnas:")
print(tess_df.columns.tolist())

print("\n📊 Valores nulos:")
tess_nulls = (tess_df.isnull().sum() / len(tess_df) * 100).sort_values(ascending=False).head(10)
print(tess_nulls)

# =================================================================
# 8. COMPARACIÓN ENTRE DATASETS
# =================================================================

print("\n" + "=" * 70)
print("COMPARACIÓN ENTRE DATASETS")
print("=" * 70)

comparison = pd.DataFrame({
    'Dataset': ['Kepler KOI', 'K2', 'TESS TOI'],
    'Filas': [len(kepler_df), len(k2_df), len(tess_df)],
    'Columnas': [len(kepler_df.columns), len(k2_df.columns), len(tess_df.columns)],
    'Porcentaje_Nulos': [
        f"{kepler_df.isnull().sum().sum() / (kepler_df.shape[0] * kepler_df.shape[1]) * 100:.2f}%",
        f"{k2_df.isnull().sum().sum() / (k2_df.shape[0] * k2_df.shape[1]) * 100:.2f}%",
        f"{tess_df.isnull().sum().sum() / (tess_df.shape[0] * tess_df.shape[1]) * 100:.2f}%"
    ]
})

display(comparison)

# =================================================================
# 9. RECOMENDACIONES PARA PREPROCESAMIENTO
# =================================================================

print("\n" + "=" * 70)
print("🎯 RECOMENDACIONES PARA PREPROCESAMIENTO")
print("=" * 70)

print("""
BASADO EN LOS PAPERS Y EL ANÁLISIS:

1. COLUMNAS A ELIMINAR (Kepler KOI):
   ✗ Identificadores: kepid, kepoi_name, kepler_name, rowid
   ✗ Columnas de score: koi_score
   ✗ Columnas vacías o con >80% nulos
   ✗ Columnas de error (err1, err2) - opcional según paper

2. COLUMNA OBJETIVO:
   ✓ Kepler: 'koi_disposition' → Codificar a binario
     - "CONFIRMED" = 1 (Planeta confirmado)
     - "CANDIDATE" = 0 (Candidato)
     - Eliminar "FALSE POSITIVE"

3. MANEJO DE VALORES NULOS:
   ✓ Imputar con la MEDIA de cada columna (método del paper)
   ✓ Considerar eliminar columnas con >50% de nulos

4. NORMALIZACIÓN:
   ✓ Usar StandardScaler (método del paper)
   ✓ Aplicar DESPUÉS de dividir train/test

5. FEATURES MÁS IMPORTANTES (según papers):
   ⭐ Parámetros orbitales: koi_period, koi_duration, koi_depth
   ⭐ Parámetros estelares: koi_steff, koi_srad
   ⭐ Parámetros planetarios: koi_prad, koi_teq

6. DIVISIÓN DE DATOS:
   ✓ 70% entrenamiento / 30% prueba (método del paper)
   ✓ Usar stratify para mantener proporción de clases

PRÓXIMOS PASOS:
→ Crear script de preprocesamiento
→ Implementar extracción de características con TSFRESH
→ Entrenar modelo LightGBM o Ensemble (Stacking)
""")

print("\n✅ ANÁLISIS EXPLORATORIO COMPLETADO")
print("=" * 70)